# Building a Collaborative-Filtering Recommendation Sytem from Scratch
Music recommendation system can be broadly categorised into three categories: content-based, collaborative filtering, and a hybrid of the two. In a content-based recommendation system, a user profile is built based on their likes and dislikes. The algorithm then analyses the items and their descriptions to determine the best possible choice for the user$^1$. On the other hand, collaborative filtering has two approaches: user-based and item-based. A user-based collaborative filtering recommendation system “relies on the fact each person belongs in a group of similarly behaving individuals” $^2$. By taking into account the actions of the group, the individuals behaviour can be predicted. In an item-based approach, the similarities between the products are computed, then the system will infer likes and dislikes based on whether or not the products are similar to what they have already purchased. In a hybrid approach, most combine collaborative filtering with some from of content-based filtering with a certain weight $^3$.

We will be testing our implementation on [The Echo Nest Taste Profile Subset](http://millionsongdataset.com/tasteprofile/), in the challenge introduced at the [Million Song Dataset Challenge](https://www.kaggle.com/c/msdchallenge/data). We will be working on the smaller subset, where we have information on $10000$ users, across $49029$ songs, provided by $131039$ triplets (under data/valid_triplets_hidden.txt and data/valid_triplets_visible.txt. For computational reasons, we will not be working on the larger susbset, containing $100000$ users, $157251$ songs, and $1319894$ triplets (under data/test_triplets_hidden.txt and data/test_triplets_visible.txt). However, with a more powerful computer, applying the same algorithm is not be an issue.

### Notation ###
From this point on, we will be using the algorithm introdcued in [this research paper](https://dl.acm.org/doi/10.1145/963770.963776). The notation will be consistent of that in the paper. Namely, $n$ will denote the number of distinct users, $m$ the number of distinct items, $R$ the $n \times m$ *user-item matrix*, where the $R_{i, j}$ corresponds to how many times user $i$ played the song $j$. Because noramlization of each row improved the result in all cases in the research paper, all rows are normalized, dividing each item by the Euclidean norm. The matrix $M$ represents the *similarity matrix*, where $M_{i,j}$ is the similairity measure between song $i$ and $j$ if $i \neq j$ and $0$ otherwise. Finally the vector $U$ generally denotes the users' basket, i.e. the items that they bought. 

In [322]:
import numpy as np
from scipy.sparse import *
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [327]:
file_triplet = 'data/evaldata/valid_triplets_visible.txt'

In [328]:
def load_users_songs(file_triple):
    """ given file_triplet of the form: user_id \t song_id \t play_times \n
        returns two dictionaries, associates users and songs to their index
        returns two lists,        associates indexes to the users and songs"""
    
    unique_users, unique_songs  = dict(), dict()
    users, songs = [], []
    
    with open(file_triplet, "r") as f:
        i, j = 0, 0
        for l in f:
            
            user_id, song_id, _ = l.split('\t')
            
            if user_id not in unique_users:
                unique_users[user_id] = i
                users.append(user_id)
                i += 1
                
            if song_id not in unique_songs:
                unique_songs[song_id] = j
                songs.append(song_id)
                j += 1
                  
    return unique_users, unique_songs, users, songs

In [329]:
def build_info(file_triplet, user_dict, song_dict):
    """
    given the file_triplet and the mapping between the user_id and the indexes, 
    returns data, row_ind, col_ind, n, m
    where 
    """
    # user_id \t song \t no. of times
    n, m = len(user_dict), len(song_dict)
    
    # build this sparse matrix
    data, row_ind, col_ind = [], [], []
    with open(file_triplet, "r") as f:
        prev_user = ''
        curr_data = []

        for l in f:
            user_id, song_id, freq = l.split('\t')
            row_ind.append(user_dict[user_id])
            col_ind.append(song_dict[song_id])
            curr_data.append(int(freq.strip('\n')))
            
            # normalization
            if prev_user != user_id:
                data += [i / np.linalg.norm(curr_data) for i in curr_data]
                curr_data = []
                
            prev_user = user_id
            
    data += [i / sum(curr_data) for i in curr_data]
    
    return data, row_ind, col_ind, n, m

In [330]:
user_dict, song_dict, users, songs = load_users_songs(file_triplet)
data, row_ind, col_ind, n, m = build_info(file_triplet, user_dict, song_dict)

In [333]:
def build_R_col(data, row_ind, col_ind, n, m):
    return csc_matrix((data, (row_ind, col_ind)), shape=(n,m))

def build_R_row(data, row_ind, col_ind, n, m):
    return csr_matrix((data, (row_ind, col_ind)), shape=(n,m))

In [334]:
R_row = build_R_row(data, row_ind, col_ind, n, m)
R_col = build_R_row(data, row_ind, col_ind, n, m)

In [335]:
def build_M_cos(R_col):
    M = cosine_similarity(R_col.transpose(), dense_output = False)
    for i in range(M.shape[0]):
        M[i,i] = 0
    return M

In [336]:
M_cos = build_M_cos(R_col)

Now that we have built the similarity matrix $M$ using the cosine similarity measure, we will be building the similarity matrix $M$ using the conditional probability measure using the formula introduced in section 4.1.1.2.

In [191]:
def song_user_dict(triple_file, song_dict, user_dict):
    # res[song_index] = list of user_index that bought the song
    res = dict()
    
    with open(file_triplet, "r") as f:
        for l in f:
            user_id, song_id, _ = l.split('\t')
            if song_dict[song_id] in res:
                res[song_dict[song_id]].append(user_dict[user_id])
            else:
                res[song_dict[song_id]] = [user_dict[user_id]]
                
    return res

In [192]:
song_user = song_user_dict(file_triplet, song_dict, user_dict)

In [15]:
def intersection(A, B):
    # https://stackoverflow.com/questions/29201260/a-fast-way-to-find-the-number-of-elements-in-list-intersection-python
    fst, snd = (A, B) if len(A) < len(B) else (B, A)
    return len(set(fst).intersection(snd))

In [116]:
# this takes a lot of time computationally, have already run it so we can load it with pickle
def build_M_prob(song_user):
    data, row_ind, col_ind = [], [], []
    m = len(song_user)
    
    for i in range(m):
        for j in range(m):
            if i != j:
                freq_ij = intersection(song_user[i], song_user[j])
                if freq_ij != 0:
                    data.append(freq_ij)
                    row_ind.append(i)
                    col_ind.append(j)
                    
    return csc_matrix((data, (row_ind, col_ind)), shape=(m,m))

In [117]:
# M_prob = build_M_prob(song_user)
# file_to_write = open("M_prob.pickle", "wb")
# pickle.dump(M_prob, file_to_write)

In [337]:
M_prob = pickle.load(open("M_prob.pickle", "rb"))

We apply the model, using the psuedo-code on how to apply the similarity matirx $M$ we have just created (Algorithm 4.2).

In [338]:
def recommend(user_id, user_dict, songs, R_row, M, k = 10):
    
    U = R_row.getrow(user_dict[user_id]).transpose()
    x = M.dot(U).getcol(0).transpose().todense().A[0]
    
    for j in range(m):
        if j in U.indices:
            x[j] = 0
    
    return set(songs[j] for j in x.argsort()[:-k-1:-1])

In [339]:
# example of 10 recommendations
recommend("0007140a3796e901f3190f12e9de6d7548d4ac4a", user_dict, songs, R_row, M_cos)

{'SOIUMKH12A6D4F91A4',
 'SOPTXHV12AB01889F1',
 'SOQOAGV12A8C132F7D',
 'SOSATQZ12AB017DF2F',
 'SOUCKTI12AB017DF15',
 'SOUGPVV12AB0186397',
 'SOUOULX12A8C13A4F8',
 'SOUTTSQ12A6D4F43FA',
 'SOWOSHB12AF72A237C',
 'SOXQMFT12A58A7AA88'}

Now, we can compute the accuracy of the system. We built recommendations for each user, associating each user_id with a set of song_ids that were recommended.

In [340]:
# recommendations_cos = dict()
# for u in users:
#     recommendations_cos[u] = recommend(u, user_dict, songs, R_row, M_cos)

In [341]:
# recommendations_prob = dict()
# for u in users:
#    recommendations_prob[u] = recommend(u, user_dict, songs, R_row, M_prob)

In [342]:
# file_to_write = open("recommendations_cos.pickle", "wb")
# pickle.dump(recommendations_cos, file_to_write)

In [343]:
# file_to_write = open("recommendations_prob.pickle", "wb")
# pickle.dump(recommendations_prob, file_to_write)

In [344]:
recommendations_cos = pickle.load(open("recommendations_cos.pickle", "rb"))
recommendations_prob = pickle.load(open("recommendations_prob.pickle", "rb"))

In [345]:
def valid_set(hidden_triplet, users):
    res = {u : set() for u in users}
    with open(hidden_triplet, "r") as f:
        for l in f:
            user_id, song_id, _ = l.split('\t')
            res[user_id].add(song_id)
    return res

In [346]:
true_result = valid_set("data/evaldata/valid_triplets_hidden.txt", users)

To evaluate the accuracy of our model, we use the [Jaccard index](https://en.wikipedia.org/wiki/Jaccard_index) where 
$$
    J(A, B) = \frac{|A \cap B|}{|A \cup B|}
$$
It is used to compute the similarity of two sample sets.

In [347]:
import random

In [348]:
J_ind_cos, J_ind_prob, J_ind_random = [], [], []

for user, res in true_result.items():
    recs_cos = recommendations_cos[user]
    recs_prob = recommendations_prob[user]
    recs_random = set(random.sample(songs_cpy, 10))
    
    J_ind_cos.append( len(recs_cos.intersection(res)) / len(recs_cos.union(res)) )
    J_ind_prob.append( len(recs_prob.intersection(res)) / len(recs_prob.union(res)) )
    J_ind_random.append( len(recs_random.intersection(res)) / len(recs_random.union(res)) )
    
performance_cos =  np.array(J_ind_cos).mean()
performance_prob = np.array(J_ind_prob).mean()
performance_random = np.array(J_ind_random).mean()
    
print("Performance of Cosine-Based Similarity-Measure:            ", np.array(Jaccard_indices_cos).mean())
print("Performance of Conditional Probability Similarity-Measure: ", np.array(Jaccard_indices_prob).mean())
print("Performance of Random Recommendation:                      ", np.array(Jaccard_indices_prob).mean())
print("Cosine performs                  {}x better than random.".format(round(performance_cos / performance_random, 2)))
print("Conditional probability performs {}x better than random.".format(round(performance_prob / performance_random, 2)))

Performance of Cosine-Based Similarity-Measure:             0.012384627325967344
Performance of Conditional Probability Similarity-Measure:  0.0373200247213094
Performance of Random Recommendation:                       0.0373200247213094
Cosine performs                  81.75x better than random.
Conditional probability performs 246.36x better than random.
